In [1]:
import serial
import codecs
import time
import numpy as np

In [2]:
#check status to find current monochromator wavelength if above or below home
def check_status(wait_time=1):
    ser = serial.Serial('COM4', 
                            baudrate = 9600, 
                            timeout = None,
                            xonxoff = True,
                            parity = serial.PARITY_NONE,
                            stopbits = serial.STOPBITS_ONE,
                            bytesize = serial.EIGHTBITS,
                            )
    ser.close()
    time.sleep(wait_time) #gives command time to readout full message from serial reciever
    ser.open()
    ser.write(b'] \r'); #check limit status
    s = ser.read_until(size=None) #reads output from serial until there is no more data transmitted
    ser.close()
    stat_now = codecs.decode(s) #decodes info from serial to m-23000 to confirm movement
    time.sleep(wait_time)
    stat_now = int(str(stat_now[4:])) #should slice output to only be the interger, not ]    0 
    return stat_now #updates value for stat_now when calling stat_now = check_status()

In [3]:
#Function to home monochromator to 631.26nm
def home():
    ser = serial.Serial('COM4', 
                            baudrate = 9600, 
                            timeout = None,
                            xonxoff = True,
                            parity = serial.PARITY_NONE,
                            stopbits = serial.STOPBITS_ONE,
                            bytesize = serial.EIGHTBITS,
                            )
    ser.close()
    #initialize connection of program and monochromator
    ser.open()                            
    ser.write(b' \r'); 
    s = ser.read_until(size=None) 
    ser.close()
    print("communication initialized")

    #enable home circuit to configure to home wavelength
    ser.open()
    ser.write(b'A8 \r'); 
    s=ser.read_until(size=None)
    ser.close()
    print("home circuit enabled")

    #check the limit status before starting movement, so the code know to scan up or down
    stat_now = check_status()
    print("initial limit status is:",stat_now)
    
    #raise Exception if scan controller already at home

    if stat_now < 32: #above home stat_now=0, above home and moving stat_now=2
        ser.open()
        ser.write(b'm-23000 \r'); #move at constant vel. of 23KHz decreasing wavelength
        s=ser.read_until(size=None)
        ser.close() 
        print("decreasing wavelength at a rate of 23KHz")

        while stat_now < 32: #once scan passes home stat_now should switch to 34
            stat_now = check_status() 
            print("limit status is:",stat_now)

            if stat_now > 32: #below home stat_now=32, below home and moving stat_now=34
                ser.open()
                ser.write(b'@ \r'); #soft stop
                s = ser.read_until(size=None)
                ser.close()
                print("scan controller stopped")

                time.sleep(0.8)
                ser.open()
                ser.write(b'-108000 \r'); #turns motor for 3 rev., subtracts 12nm
                s = ser.read_until(size=None)
                ser.close()
                print("decreasing wavelength for 3 revolutions")

                time.sleep(2)
                ser.open()
                ser.write(b'+72000 \r'); #turns motor for 2 rev., removes backlash adds 8nm
                s = ser.read_until(size=None)
                ser.close()
                print("increasing wavelength for 2 revolutions")
                
                time.sleep(1)
                ser.open()
                ser.write(b'A24 \r'); #enable high accuracy circuit for fine movement
                s = ser.read_until(size=None)
                ser.close()
                print("high accuracy circuit enabled")

                ser.open()
                ser.write(b'F4500,0 \r'); #find edge of home flag at 1000 steps/sec
                s = ser.read_until(size=None)
                ser.close()
                print("finding edge of home flag at 4500KHz")

                time.sleep(35) #time it takes to complete F1000,0 movement
                ser.open()
                ser.write(b'@ \r'); #soft stop
                s = ser.read_until(size=None)
                ser.close()
                print("scan controller stopped")

                ser.open()
                ser.write(b'A0 \r'); #disables home circuit
                s = ser.read_until(size=None)
                ser.close()
                print("disabled home circuit")

                ser.open()
                ser.write(b'P \r'); #exit program
                s = ser.read_until(size=None)
                ser.close()
                print("exited program ")
                print("finished homing, now at 631.1nm")

    if stat_now > 0: #below home wavelength stat_now=32, below home and moving stat_now=34
        ser.open()
        ser.write(b'm+23000 \r')
        s = ser.read_until(size=None)
        pos = codecs.decode(s)
        ser.close()
        print("increasing wavelength at a rate of 23KHz ")

        while stat_now > 2: #once scan passes home stat_now should switch to 2
            stat_now = check_status()
            print("limit status is:",stat_now)
            
            if stat_now < 32: #above home wavelength stat_now=0, above home and moving stat_now=2
                ser.open()
                ser.write(b'@ \r'); #soft stop
                s = ser.read_until(size=None)
                stop = codecs.decode(s)
                print("scan controller stopped")
                ser.close()

                time.sleep(2)    
                ser.open()
                ser.write(b'-108000 \r'); #decreasing wavelength for 3 motor revolutions
                s = ser.read_until(size=None)
                ser.close()
                print("decrease wavelength for 3 revolutions")

                time.sleep(1)
                ser.open()
                ser.write(b'+72000 \r'); #removes backlash
                s = ser.read_until(size=None)
                ser.close()
                print("increase wavelength for 2 revolutions")

                ser.open()
                ser.write(b'A24 \r'); #enable high accuracy circuit
                s = ser.read_until(size=None)
                ser.close()
                print("high accuracy circuit enabled")

                ser.open()
                ser.write(b'F4500,0 \r'); #find edge of home flag at 1000 steps/sec
                s = ser.read_until(size=None)
                ser.close()
                print("finding edge of home flag at 4500KHz ")

                time.sleep(35)
                ser.open()
                ser.write(b'@ \r'); #soft stop
                s = ser.read_until(size=None)
                ser.close()
                print("scan controller stopped")

                ser.open()
                ser.write(b'A0 \r'); #disable home circuit
                s = ser.read_until(size=None)
                ser.close()
                print("disabled home circuit")

                ser.open()
                ser.write(b'P \r'); #exit program
                s = ser.read_until(size=None)
                ser.close()
                print("exited program ")
                print("finished homing, now at 631.1nm")

In [4]:
home()

communication initialized
home circuit enabled
initial limit status is: 0
decreasing wavelength at a rate of 23KHz
limit status is: 2
limit status is: 2
limit status is: 34
scan controller stopped
decreasing wavelength for 3 revolutions
increasing wavelength for 2 revolutions
high accuracy circuit enabled
finding edge of home flag at 4500KHz
scan controller stopped
disabled home circuit
exited program 
finished homing, now at 631.1nm
increasing wavelength at a rate of 23KHz 


ValueError: invalid literal for int() with base 10: ''

#emergency manual stop 
import serial

ser = serial.Serial('COM4', 
                    baudrate = 9600, 
                    timeout = None,
                    xonxoff = True,
                    parity = serial.PARITY_NONE,
                    stopbits = serial.STOPBITS_ONE,
                    bytesize = serial.EIGHTBITS,
                    )

ser.close()

ser.open()
ser.write(b'@ \r'); #soft stop
s = ser.read_until(size=None)
ser.close()
print("input manually")

    #else: #else 
        #print("error reading limit status, could not determine distance from home." )
        #ser.open()
        #ser.write(b'@ \r'); #soft stop
        #s = ser.read_until(size=None)
        #ser.close()
        #print("scan controller stopped")

        #ser.open()
        #ser.write(b'P \r'); #exit program
        #s = ser.read_until(size=None)
        #ser.close()
        #print("exited program")
        #raise Exception("error reading limit status, could not determine distance from home.")
        #exit

#tests if connction is viable and sends response to user
def connect():
    if ser.is_open(): #checks for open comm port used already by terminal
        print("Connection already established")
    else:
    #defines port and connection for scan controller
        ser = serial.Serial('COM4', 
                            baudrate = 9600, 
                            timeout = None,
                            xonxoff = True,
                            parity = serial.PARITY_NONE,
                            stopbits = serial.STOPBITS_ONE,
                            bytesize = serial.EIGHTBITS,
                            )
        ser.close()
        print("connection established")
    raise Exception 
    print("Error, not able to make connection ") #lets user know connection is not ready
    
    return connect()

#serial data for connection to monochromator
def ser():
    ser = serial.Serial('COM4', 
                            baudrate = 9600, 
                            timeout = None,
                            xonxoff = True,
                            parity = serial.PARITY_NONE,
                            stopbits = serial.STOPBITS_ONE,
                            bytesize = serial.EIGHTBITS,
                            )
    ser.close()
    print("connection established")
    
    return ser()